In [4]:
import time
from lib.ev3 import EV3


# センサーとモーターの通信ポートの定義.
touch_port = EV3.PORT_2
color_port = EV3.PORT_3
lmotor_port = EV3.PORT_B
rmotor_port = EV3.PORT_C

# センサーとモーターの設定.
ev3 = EV3()
ev3.motor_config(lmotor_port, EV3.LARGE_MOTOR)
ev3.motor_config(rmotor_port, EV3.LARGE_MOTOR)
ev3.sensor_config(touch_port, EV3.TOUCH_SENSOR)
ev3.sensor_config(color_port, EV3.COLOR_SENSOR)

# タッチセンサーを押して離すとスタート.
ev3.lcd_draw_string('Push to start.', 0)
while not ev3.touch_sensor_is_pressed(touch_port):
    pass
while ev3.touch_sensor_is_pressed(touch_port):
    pass

ev3.lcd_draw_string('Go!', 0)

# 制御ループ
while True:
    # タッチセンサーが押されたら終了.
    if ev3.touch_sensor_is_pressed(touch_port):
        break
    # P制御でステアリング値を計算.
    white = 70  # 白面上の反射値.
    black = 5   # 黒面上の反射値.
    midpoint = (white + black) / 2  # 平均値を計算.
    color = ev3.color_sensor_get_reflect(color_port)
    steer = 0.8 * (color - midpoint)  # p * (測定値 - 平均値)
    # EV3へ制御値を送信.
    ev3.motor_steer(lmotor_port, rmotor_port, 10, int(steer))

# モーターの停止.
ev3.lcd_draw_string('Stop.', 0)
ev3.motor_steer(lmotor_port, rmotor_port, 0, 0)
# 終了処理.
ev3.close()